In [1]:
import json

import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

import pandas as pd
import openclsim.core as core
import openclsim.model as model

import datetime

import numpy as np

from plot import vessel_planning

from vo_colos.utils.object_registry import SiteRegistry, VesselRegistry
from vo_colos.utils.object_registry import ClassInspect

from vo_colos.utils.base_task import SimulationFactory

In [2]:
# with open("./config subcycle.json", 'r') as fp:
#     config = json.load(fp)

config = {
    "tag": [],
    "engine": "colos",
    "config": {
        "simulationProperties": {},
        "activities": [
            {
                "name": "while",
                "class": "WhileActivity",
                "properties": {
                    "sub_process": {
                        "name": "sequence",
                        "class": "SequenceActivity",
                        "properties": {
                            "id": "bff7f5b6-5340-46dc-91bf-a15ee668817d",
                            "sub_processes": [
                                {
                                    "name": "sailing empty",
                                    "class": "MoveActivity",
                                    "properties": {
                                        "id": "2a41cbf9-c798-430c-be5f-fe60808c7e0f",
                                        "mover": "Hopper",
                                        "destination": "Origin"
                                    }
                                },
                                {
                                    "name": "while_subcycle",
                                    "class": "WhileActivity",
                                    "properties": {
                                        "id": "c6ea73c5-3c6b-4b80-b471-1076b4d9443d",
                                        "condition_event": [
                                            {
                                                "or": [
                                                    {
                                                        "type": "container",
                                                        "state": "full",
                                                        "concept": "Hopper"
                                                    },
                                                    {
                                                        "type": "container",
                                                        "state": "empty",
                                                        "concept": "Origin"
                                                    }
                                                ]
                                            }
                                        ],
                                        "sub_process": {
                                            "name": "subcycle",
                                            "class": "SequenceActivity",
                                            "properties": {
                                                "id": "335dfb3c-8392-40cd-8875-25c74272a71f",
                                                "sub_processes": [
                                                    {
                                                        "name": "Basic A",
                                                        "class": "BasicActivity",
                                                        "properties": {
                                                            "id": "a70f524c-f624-4ad6-922c-fff9e9a684f0",
                                                            "duration": 360
                                                        }
                                                    },
                                                    {
                                                        "name": "Basic B",
                                                        "class": "BasicActivity",
                                                        "properties": {
                                                            "id": "140330ce-fa06-4876-9dc1-666305422ba4",
                                                            "duration": 360
                                                        }
                                                    },
                                                    {
                                                        "name": "Basic C",
                                                        "class": "BasicActivity",
                                                        "properties": {
                                                            "id": "dcc357db-6b47-45dc-bdb9-b17f21759fc7",
                                                            "duration": 360
                                                        }
                                                    },
                                                    {
                                                        "name": "Loading",
                                                        "class": "ShiftAmountActivity",
                                                        "properties": {
                                                            "id": "d182f529-2027-4284-a763-3664158403a5",
                                                            "origin": "Origin",
                                                            "destination": "Hopper",
                                                            "processor": "Hopper",
                                                            "amount": 10,
                                                            "duration": 720
                                                        }
                                                    }
                                                ]
                                            }
                                        }
                                    }
                                },
                                {
                                    "name": "Sailing empty",
                                    "class": "MoveActivity",
                                    "properties": {
                                        "id": "59f403c9-688a-4bfd-bf39-2044b3e6c96b",
                                        "mover": "Hopper",
                                        "destination": "Destination"
                                    }
                                },
                                {
                                    "name": "unloading",
                                    "class": "ShiftAmountActivity",
                                    "properties": {
                                        "id": "274770ea-59a6-4a23-b052-ea68a85c564b",
                                        "origin": "Hopper",
                                        "destination": "Destination",
                                        "processor": "Hopper",
                                        "amount": 100,
                                        "duration": 3598
                                    }
                                }
                            ]
                        }
                    }
                }
            }
        ],
        "vessels": {
            "Hopper": {
                "class": "TestHopper",
                "properties": {
                    "v": 1,
                    "lat": 50,
                    "lon": 6,
                    "level": 0,
                    "v_full": 10,
                    "v_empty": 10,
                    "capacity": 100,
                    "loading_rate": 1,
                    "nr_resources": 1,
                    "store_capacity": 1,
                    "unloading_rate": 1,
                    "load_manoeuvring": 1,
                    "unload_manoeuvring": 1
                }
            }
        },
        "sites": {
            "Origin": {
                "class": "TestSite",
                "properties": {
                    "lat": 50,
                    "lon": 6,
                    "level": 1000,
                    "capacity": 1000,
                    "nr_resources": 1,
                    "store_capacity": 1
                }
            },
            "Destination": {
                "class": "TestSite",
                "properties": {
                    "lat": 50,
                    "lon": 7,
                    "level": 0,
                    "capacity": 1000,
                    "nr_resources": 1,
                    "store_capacity": 1
                }
            }
        },
        "graph": {
            "nodes": [],
            "edges": []
        }
    }
}


In [3]:
print(json.dumps(config, indent=4))

{
    "tag": [],
    "engine": "colos",
    "config": {
        "simulationProperties": {},
        "activities": [
            {
                "name": "while",
                "class": "WhileActivity",
                "properties": {
                    "sub_process": {
                        "name": "sequence",
                        "class": "SequenceActivity",
                        "properties": {
                            "id": "bff7f5b6-5340-46dc-91bf-a15ee668817d",
                            "sub_processes": [
                                {
                                    "name": "sailing empty",
                                    "class": "MoveActivity",
                                    "properties": {
                                        "id": "2a41cbf9-c798-430c-be5f-fe60808c7e0f",
                                        "mover": "Hopper",
                                        "destination": "Origin"
                                    }
             

In [4]:
class message:
    id = 1
    task = {
        "id": 1,
        "engine": "colos",
        "project_id": "1234567890",
        "tag": ["testsimulation"],
        "config": config["config"],
    }
    status = "new"

sim = SimulationFactory(message)

In [5]:
sim.simulate()

init
level: 1000
completed init
init
level: 0
completed init
init
level: 0
completed init
{'sub_process': {'name': 'sequence', 'class': 'SequenceActivity', 'properties': {'id': 'bff7f5b6-5340-46dc-91bf-a15ee668817d', 'sub_processes': [{'name': 'sailing empty', 'class': 'MoveActivity', 'properties': {'id': '2a41cbf9-c798-430c-be5f-fe60808c7e0f', 'mover': 'Hopper', 'destination': 'Origin'}}, {'name': 'while_subcycle', 'class': 'WhileActivity', 'properties': {'id': 'c6ea73c5-3c6b-4b80-b471-1076b4d9443d', 'condition_event': [{'or': [{'type': 'container', 'state': 'full', 'concept': 'Hopper'}, {'type': 'container', 'state': 'empty', 'concept': 'Origin'}]}], 'sub_process': {'name': 'subcycle', 'class': 'SequenceActivity', 'properties': {'id': '335dfb3c-8392-40cd-8875-25c74272a71f', 'sub_processes': [{'name': 'Basic A', 'class': 'BasicActivity', 'properties': {'id': 'a70f524c-f624-4ad6-922c-fff9e9a684f0', 'duration': 360}}, {'name': 'Basic B', 'class': 'BasicActivity', 'properties': {'id': '1

KeyError: 'condition_event'

In [ ]:
objects = list(sim.vessels)
objects.extend(sim.locations)
# objects.extend(sim.activities)


activities = []
for obj in objects:
    activities.extend(set(obj.log["Message"]))

activities = list(set(activities))

vessel_planning(objects, activities)